In [1]:
import pandas as pd
import numpy as np
from late_fusion import *
import itertools

c:\Users\Justin\Documents\JKU\2024WS\multimedia-search-and-retrieval\.venv\Lib\site-packages\tqdm_joblib\__init__.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
feature_files = {
    'mfcc_bow': 'dataset/id_mfcc_bow_mmsr.tsv',
    #'mfcc_stats': 'dataset/id_mfcc_stats_mmsr.tsv',

    #'blf_correlation' : 'dataset/id_blf_correlation_mmsr.tsv',
    #'blf_spectral': 'dataset/id_blf_spectral_mmsr.tsv',
    #'blf_spectralcontrast': 'dataset/id_blf_spectralcontrast_mmsr.tsv',
    #'blf_deltaspectral': 'dataset/id_blf_deltaspectral_mmsr.tsv',
    #'blf_logfluc': 'dataset/id_blf_logfluc_mmsr.tsv',
    #'blf_vardeltaspectral': 'dataset/id_blf_vardeltaspectral_mmsr.tsv',

    #'ivec256': 'dataset/id_ivec256_mmsr.tsv',
    #'ivec512': 'dataset/id_ivec512_mmsr.tsv',
    #'ivec1024': 'dataset/id_ivec1024_mmsr.tsv',topKasdasdassadasdasdasdasd

    'musicnn': 'dataset/id_musicnn_mmsr.tsv',

    'lyrics_bert': 'dataset/id_lyrics_bert_mmsr.tsv',
    #'lyrics_tf-idf': 'dataset/id_lyrics_tf-idf_mmsr.tsv',
    #'lyrics_word2vec': 'dataset/id_lyrics_word2vec_mmsr.tsv',

    'resnet': 'dataset/id_resnet_mmsr.tsv',
    #'vgg': 'dataset/id_vgg19_mmsr.tsv',
    #'incp': 'dataset/id_incp_mmsr.tsv',

    'artist': 'dataset/id_information_mmsr.tsv',
}


In [ ]:
retrieval = MultiModalLateFusionRetrieval()
retrieval.load_and_normalize_features(feature_files)

feature_weights = {    
    'mfcc_bow':                 0.6,
    #'mfcc_stats':              

    #'blf_correlation' :        
    #'blf_spectral':            
    #'blf_spectralcontrast':    
    #'blf_deltaspectral':       
    #'blf_logfluc':             
    #'blf_vardeltaspectral':    

    #'ivec256':                 
    #'ivec512':                 
    #'ivec1024':                

    'musicnn':                  0.4,

    'lyrics_bert':              0.5,
    #'lyrics_tf-idf':           
    #'lyrics_word2vec':         

    'resnet':                   0.3,
    #'vgg':                     
    #'incp':                    

    'artist':                   0.7,
}
infos = pd.read_csv("dataset/id_information_mmsr.tsv", sep="\t")

song_ids = infos['id'].tolist()
results = retrieval.batch_similarity_search(song_ids, top_k=100, fusion_method="weighted_average", weights=feature_weights)

Using 4 cores for batch similarity search.


Processing batch similarity search: 100%|██████████| 5148/5148 [00:06<00:00, 762.65it/s]


In [4]:
results_df = pd.DataFrame(results)
results_df.to_csv('./predictions/ui/rets_late_fusion_100.csv', sep=",", index=False, header=True)

In [ ]:
inter_matr = np.zeros((len(infos), len(infos)))

song_id_to_index = {song_id: idx for idx, song_id in enumerate(song_ids)}

for idx, row in tqdm(results_df.iterrows(), desc="generate matrix", total=len(results_df)):
    source = row["source_id"]
    target = row["target_id"]
    inter_matr[song_id_to_index[source], song_id_to_index[target]] = row["similarity"]

np.savetxt(f"./predictions/rets_late_fusion_100_matrix.csv", inter_matr, delimiter=",")

generate matrix: 100%|██████████| 2574000/2574000 [01:46<00:00, 24239.84it/s]
